<h2 align='center' style='color:blue'>Dropout Regularization In Deep Neural Network</h2>

This is a dataset that describes sonar chirp returns bouncing off different services. The 60 input variables are the strength of the returns at different angles. It is a **binary classification problem** that requires a model to differentiate rocks from metal cylinders.

Dataset information: https://archive.ics.uci.edu/ml/datasets/Connectionist+Bench+(Sonar,+Mines+vs.+Rocks)
Download it from here: https://archive.ics.uci.edu/ml/machine-learning-databases/undocumented/connectionist-bench/sonar/sonar.all-data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("./sonar_dataset.csv", header=None)
df.sample(5)

,0,1,2,3,4,5,6,7,8,9,...,51,52,53,54,55,56,57,58,59,60
132,0.0968,0.0821,0.0629,0.0608,0.0617,0.1207,0.0944,0.4223,0.5744,0.5025,...,0.0073,0.0081,0.0303,0.0190,0.0212,0.0126,0.0201,0.0210,0.0041,M
170,0.0134,0.0172,0.0178,0.0363,0.0444,0.0744,0.0800,0.0456,0.0368,0.1250,...,0.0094,0.0048,0.0047,0.0016,0.0008,0.0042,0.0024,0.0027,0.0041,M
157,0.0201,0.0178,0.0274,0.0232,0.0724,0.0833,0.1232,0.1298,0.2085,0.2720,...,0.0131,0.0049,0.0104,0.0102,0.0092,0.0083,0.0020,0.0048,0.0036,M
52,0.0087,0.0046,0.0081,0.0230,0.0586,0.0682,0.0993,0.0717,0.0576,0.0818,...,0.0052,0.0038,0.0079,0.0114,0.0050,0.0030,0.0064,0.0058,0.0030,R
142,0.0526,0.0563,0.1219,0.1206,0.0246,0.1022,0.0539,0.0439,0.2291,0.1632,...,0.0339,0.0149,0.0335,0.0376,0.0174,0.0132,0.0103,0.0364,0.0208,M


In [4]:
df.shape

(208, 61)

In [5]:
# check for nan values
df.isna().sum()

0     0
1     0
2     0
3     0
4     0
     ..
56    0
57    0
58    0
59    0
60    0
Length: 61, dtype: int64

In [7]:
df.columns

Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53,
       54, 55, 56, 57, 58, 59, 60],
      dtype='int64')

In [8]:
df[60].value_counts() # label is not skewed

60
M    111
R     97
Name: count, dtype: int64

In [9]:
X = df.drop(60, axis=1)
y = df[60]
y.head()

0    R
1    R
2    R
3    R
4    R
Name: 60, dtype: object

In [10]:
y = pd.get_dummies(y, drop_first=True)
y.sample(5) # R --> 1 and M --> 0

,R
164,False
113,False
37,True
122,False
20,True


In [11]:
y.value_counts()

R    
False    111
True      97
Name: count, dtype: int64

In [12]:
X.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
0,0.0200,0.0371,0.0428,0.0207,0.0954,0.0986,0.1539,0.1601,0.3109,0.2111,...,0.0232,0.0027,0.0065,0.0159,0.0072,0.0167,0.0180,0.0084,0.0090,0.0032
1,0.0453,0.0523,0.0843,0.0689,0.1183,0.2583,0.2156,0.3481,0.3337,0.2872,...,0.0125,0.0084,0.0089,0.0048,0.0094,0.0191,0.0140,0.0049,0.0052,0.0044
2,0.0262,0.0582,0.1099,0.1083,0.0974,0.2280,0.2431,0.3771,0.5598,0.6194,...,0.0033,0.0232,0.0166,0.0095,0.0180,0.0244,0.0316,0.0164,0.0095,0.0078
3,0.0100,0.0171,0.0623,0.0205,0.0205,0.0368,0.1098,0.1276,0.0598,0.1264,...,0.0241,0.0121,0.0036,0.0150,0.0085,0.0073,0.0050,0.0044,0.0040,0.0117
4,0.0762,0.0666,0.0481,0.0394,0.0590,0.0649,0.1209,0.2467,0.3564,0.4459,...,0.0156,0.0031,0.0054,0.0105,0.0110,0.0015,0.0072,0.0048,0.0107,0.0094


In [13]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=1)

In [14]:
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,50,51,52,53,54,55,56,57,58,59
67,0.0368,0.0403,0.0317,0.0293,0.0820,0.1342,0.1161,0.0663,0.0155,0.0506,...,0.0058,0.0091,0.0160,0.0160,0.0081,0.0070,0.0135,0.0067,0.0078,0.0068
14,0.0124,0.0433,0.0604,0.0449,0.0597,0.0355,0.0531,0.0343,0.1052,0.2120,...,0.0078,0.0083,0.0057,0.0174,0.0188,0.0054,0.0114,0.0196,0.0147,0.0062
164,0.0163,0.0198,0.0202,0.0386,0.0752,0.1444,0.1487,0.1484,0.2442,0.2822,...,0.0027,0.0077,0.0026,0.0031,0.0083,0.0020,0.0084,0.0108,0.0083,0.0033
179,0.0394,0.0420,0.0446,0.0551,0.0597,0.1416,0.0956,0.0802,0.1618,0.2558,...,0.0118,0.0146,0.0040,0.0114,0.0032,0.0062,0.0101,0.0068,0.0053,0.0087
19,0.0126,0.0149,0.0641,0.1732,0.2565,0.2559,0.2947,0.4110,0.4983,0.5920,...,0.0153,0.0092,0.0035,0.0098,0.0121,0.0006,0.0181,0.0094,0.0116,0.0063


# Using Deep Learning Model

## Model without Dropout Layer

In [17]:
import tensorflow as tf
from tensorflow import keras

In [16]:
model = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

model.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 1ms/step - loss: 0.6898 - accuracy: 0.5192
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6680 - accuracy: 0.6282
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6528 - accuracy: 0.5833
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6272 - accuracy: 0.6603
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5963 - accuracy: 0.8141
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5596 - accuracy: 0.7949
Epoch 7/100
20/20 [==============================] - 0s 1ms/step - loss: 0.5134 - accuracy: 0.7692
Epoch 8/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4732 - accuracy: 0.8077
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4351 - accuracy: 0.8141
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 0.4195 - accuracy: 0.8141
Epoch 11/

Epoch 83/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0039 - accuracy: 1.0000
Epoch 84/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 85/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 86/100
20/20 [==============================] - 0s 959us/step - loss: 0.0034 - accuracy: 1.0000
Epoch 87/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 88/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0030 - accuracy: 1.0000
Epoch 89/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 90/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 91/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0028 - accuracy: 1.0000
Epoch 92/100
20/20 [==============================] - 0s 1ms/step - loss: 0.0030 - accuracy: 1.000

In [18]:
model.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 1.0386 - accuracy: 0.7500


[1.0386425256729126, 0.75]

Training Accuracy >>> Test Accuracy

In [19]:
y_pred = model.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 3ms/step
[2.2101652e-09 9.9319476e-01 8.3878171e-01 8.5629154e-06 9.9999988e-01
 9.9962670e-01 2.1767317e-01 1.0000000e+00 1.1309522e-06 9.9999994e-01]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [20]:
y_test[:10]

,R
186,False
155,False
165,False
200,False
58,True
34,True
151,False
18,True
202,False
62,True


In [21]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.71      0.89      0.79        27
        True       0.83      0.60      0.70        25

    accuracy                           0.75        52
   macro avg       0.77      0.74      0.74        52
weighted avg       0.77      0.75      0.74        52



### Model with Dropout Layer

In [22]:
modeld = keras.Sequential([
    keras.layers.Dense(60, input_dim=60, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(30, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(1, activation='sigmoid')
])

modeld.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modeld.fit(X_train, y_train, epochs=100, batch_size=8)

Epoch 1/100
20/20 [==============================] - 1s 1ms/step - loss: 0.7681 - accuracy: 0.4167
Epoch 2/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6916 - accuracy: 0.5064
Epoch 3/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6630 - accuracy: 0.5962
Epoch 4/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7234 - accuracy: 0.5449
Epoch 5/100
20/20 [==============================] - 0s 1ms/step - loss: 0.7231 - accuracy: 0.4615
Epoch 6/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6968 - accuracy: 0.5577
Epoch 7/100
20/20 [==============================] - 0s 2ms/step - loss: 0.7040 - accuracy: 0.5064
Epoch 8/100
20/20 [==============================] - 0s 2ms/step - loss: 0.6798 - accuracy: 0.5705
Epoch 9/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6877 - accuracy: 0.5192
Epoch 10/100
20/20 [==============================] - 0s 1ms/step - loss: 0.6917 - accuracy: 0.5064
Epoch 11/

20/20 [==============================] - 0s 1ms/step - loss: 0.3626 - accuracy: 0.8718
Epoch 84/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3267 - accuracy: 0.8782
Epoch 85/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3033 - accuracy: 0.8782
Epoch 86/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3285 - accuracy: 0.8526
Epoch 87/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3259 - accuracy: 0.9103
Epoch 88/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3268 - accuracy: 0.8782
Epoch 89/100
20/20 [==============================] - 0s 1ms/step - loss: 0.2939 - accuracy: 0.8974
Epoch 90/100
20/20 [==============================] - 0s 1ms/step - loss: 0.3595 - accuracy: 0.8590
Epoch 91/100
20/20 [==============================] - 0s 1ms/step - loss: 0.2869 - accuracy: 0.9038
Epoch 92/100
20/20 [==============================] - 0s 1ms/step - loss: 0.2788 - accuracy: 0.8910
Epoch 93/100


In [23]:
modeld.evaluate(X_test, y_test)

2/2 [==============================] - 0s 4ms/step - loss: 0.5003 - accuracy: 0.7500


[0.5003204345703125, 0.75]

Training Accuracy is still good but Test Accuracy Improved

In [24]:
y_pred = modeld.predict(X_test).reshape(-1)
print(y_pred[:10])

# round the values to nearest integer ie 0 or 1
y_pred = np.round(y_pred)
print(y_pred[:10])

2/2 [==============================] - 0s 2ms/step
[0.00209559 0.77124745 0.97173566 0.03042322 0.9998747  0.6652269
 0.18880348 0.9998764  0.03044393 0.99993235]
[0. 1. 1. 0. 1. 1. 0. 1. 0. 1.]


In [25]:
from sklearn.metrics import confusion_matrix , classification_report

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.71      0.89      0.79        27
        True       0.83      0.60      0.70        25

    accuracy                           0.75        52
   macro avg       0.77      0.74      0.74        52
weighted avg       0.77      0.75      0.74        52



**You can see that by using dropout layer test accuracy increased from 0.77 to 0.81**